In [95]:
import pandas as pd
from scipy.stats import chi2_contingency 
from scipy import stats
import numpy as np
import statsmodels.stats.api as sms
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [96]:
data = pd.read_csv('../data/MRI_data_2.csv')

In [97]:
data.head()

,Visit,Site,Sex,Age,APOE,MMSE,lh_bankssts_thickness,rh_bankssts_thickness,lh_caudalanteriorcingulate_thickness,rh_caudalanteriorcingulate_thickness,...,rhCortexVol,lhCerebralWhiteMatterVol,SubCortGrayVol,MaskVol,BrainSegVol-to-eTIV,MaskVol-to-eTIV,lhSurfaceHoles,rhSurfaceHoles,Subject_ID,Diagnosis
0,0,0,0,86.0,0,20.0,2.224,2.249,2.883,2.172,...,171640.3232,184191.9575,40324.0,1361906.0,0.664587,0.977432,24.0,13.0,KPOADC001_1,0
1,1,0,0,86.0,0,16.0,2.140,2.092,2.991,2.038,...,162216.0014,192981.9801,38797.0,1319352.0,0.659338,0.941138,23.0,17.0,KPOADC001_2,0
2,2,0,0,87.0,0,21.0,2.248,2.343,3.088,2.157,...,168541.5319,189605.4005,39722.0,1342921.0,0.660968,0.959201,30.0,24.0,KPOADC001_5,0
3,0,0,0,68.0,0,26.0,1.971,2.018,2.394,2.336,...,182858.7837,199432.8506,45318.0,1515148.0,0.652202,0.976865,22.0,21.0,KPOADC002_1,0
4,1,0,0,68.0,0,25.0,2.066,2.112,2.353,2.285,...,182255.2786,195086.1362,44336.0,1508917.0,0.643059,0.968317,20.0,9.0,KPOADC002_2,0


### 3. Statistical Tests

#### Chi Squared Tests

##### H₀ — that the 2 categorical variables being compared are independent of each other (no association) 
##### H₁ — that the 2 categorical variables being compared are dependent on each other (association).

##### 1. Gender and AD Diagnosis

In [98]:
data['Sex'] = data['Sex'].replace({0:'Female', 1:'Male'})
data['Diagnosis'] = data['Diagnosis'].replace({0:'AD', 1:'CTL', 2:'MCI'})  

chi = pd.crosstab(data.Sex, data.Diagnosis,margins=True)
print(chi)

obs = np.array([chi.iloc[0][0:-1].values,chi.iloc[1][0:-1].values])
chi_squared = stats.chi2_contingency(obs, correction=False)
print("\n(X^2, p, df):",chi_squared[0:3])

print("\nConclusion:")
if(chi_squared[1] < 0.05):
    print("At 5% significance level, we fail to reject Ha which means there is sufficient evidence to suggest that there is an association between AD diagnosis and the gender feature")
else:
    print("At 5% significance level, we fail to reject Ho which means there is sufficient evidence to suggest that there is no association between AD diagnosis and the gender feature")
    

Diagnosis   AD  CTL  MCI   All
Sex                           
Female     227  184  167   578
Male       117  150  164   431
All        344  334  331  1009

(X^2, p, df): (17.620430700492648, 0.00014920112198050345, 2)

Conclusion:
At 5% significance level, we fail to reject Ha which means there is sufficient evidence to suggest that there is an association between AD diagnosis and the gender feature


##### 2. APOE and AD Diagnosis

In [99]:
data['APOE'] = data['APOE'].replace({0:'E4E4', 1:'E3E3', 2:'E3E4', 3:'E2E3', 4:'E2E4', 5:'E2E2'})
data['Diagnosis'] = data['Diagnosis'].replace({0:'AD', 1:'CTL', 2:'MCI'})  

chi = pd.crosstab(data.APOE, data.Diagnosis,margins=True)
print(chi)
obs = np.array([chi.iloc[1][0:-1].values,chi.iloc[4][0:-1].values])
chi_squared = stats.chi2_contingency(obs, correction=False)
print("\n(X^2, p, df):",chi_squared[0:3])

print("\nConclusion:")
if(chi_squared[1] < 0.05):
    print("At 5% significance level, we fail to reject Ha which means there is sufficient evidence to suggest that there is an association between AD diagnosis and the APOE feature")
else:
    print("At 5% significance level, we fail to reject Ho which means there is sufficient evidence to suggest that there is no association between AD diagnosis and the APOE feature")

Diagnosis   AD  CTL  MCI   All
APOE                          
E2E2         0    0    2     2
E2E3        12   49   27    88
E2E4         3    7    6    16
E3E3       128  177  184   489
E3E4       140   89   98   327
E4E4        61   12   14    87
All        344  334  331  1009

(X^2, p, df): (33.02342663561697, 6.746119334452304e-08, 2)

Conclusion:
At 5% significance level, we fail to reject Ha which means there is sufficient evidence to suggest that there is an association between AD diagnosis and the APOE feature


##### 3. MMSE and AD Diagnosis

In [100]:
#create new catgeory based on MMSE score, q1, q2, q3, q4
data['MMSE_Cat'] = pd.cut(data['MMSE'], bins=[6,19,24.5,30], labels=['low','moderate','high'])

In [101]:
data['Diagnosis'] = data['Diagnosis'].replace({0:'AD', 1:'CTL', 2:'MCI'})  

chi = pd.crosstab(data.MMSE_Cat, data.Diagnosis,margins=True)
print(chi)

obs = np.array([chi.iloc[0][0:-1].values,chi.iloc[1][0:-1].values])
chi_squared = stats.chi2_contingency(obs, correction=True)
print("\n(X^2, p, df):",chi_squared[0:3])

print("\nConclusion:")
if(chi_squared[1] < 0.05):
    print("At 5% significance level, we fail to reject Ha which means there is sufficient evidence to suggest that there is an association between AD diagnosis and the MMSE score")
else:
    print("At 5% significance level, we fail to reject Ho which means there is sufficient evidence to suggest that there is no association between AD diagnosis and the MMSE score")
    

Diagnosis   AD  CTL  MCI   All
MMSE_Cat                      
low        130    0    6   136
moderate   120    2   34   156
high        93  332  291   716
All        343  334  331  1008

(X^2, p, df): (20.727375565610863, 3.1557862803324876e-05, 2)

Conclusion:
At 5% significance level, we fail to reject Ha which means there is sufficient evidence to suggest that there is an association between AD diagnosis and the MMSE score


### Confidence Interval

#### 1. Gender and Diagnosis

In [102]:
#95% confidence interval for difference in mean
data['Sex'] = data['Sex'].replace({'Female':0, 'Male':1})
data['Diagnosis'] = data['Diagnosis'].replace({'AD':0, 'CTL':1, 'MCI':2})  
female = data.query('Sex == 0')
male = data.query('Sex == 1')

cm = sms.CompareMeans(sms.DescrStatsW(female['Diagnosis']), sms.DescrStatsW(male['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

# print("\nGiven that the value of zero is included in the interval, this suggest that the means of the two population can be assumed equal as there is a difference between them. ")


Confidence Interval for u1-u2:  (-0.31390491144172805, -0.11180499310150058)


#### 2. APOE and Diagnosis

In [103]:
#95% confidence interval for difference in mean
data['APOE'] = data['APOE'].replace({'E4E4':0, 'E3E3':1, 'E3E4':2, 'E2E3':3, 'E2E4':4, 'E2E2':5})
e4e4 = data.query('APOE == 0')
e3e3 = data.query('APOE == 1')
e3e4 = data.query('APOE == 2')
e2e3 = data.query('APOE == 3')

print("\nE4E4 vs E3E3")
cm = sms.CompareMeans(sms.DescrStatsW(e4e4['Diagnosis']), sms.DescrStatsW(e3e3['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nE4E4 vs E3E4")
cm = sms.CompareMeans(sms.DescrStatsW(e4e4['Diagnosis']), sms.DescrStatsW(e3e4['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nE4E4 vs E2E3")
cm = sms.CompareMeans(sms.DescrStatsW(e4e4['Diagnosis']), sms.DescrStatsW(e2e3['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nE3E3 vs E3E4")
cm = sms.CompareMeans(sms.DescrStatsW(e3e3['Diagnosis']), sms.DescrStatsW(e3e4['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nE3E3 vs E2E3")
cm = sms.CompareMeans(sms.DescrStatsW(e3e3['Diagnosis']), sms.DescrStatsW(e2e3['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nE3E4 vs E2E3")
cm = sms.CompareMeans(sms.DescrStatsW(e3e4['Diagnosis']), sms.DescrStatsW(e2e3['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))


E4E4 vs E3E3
Confidence Interval for u1-u2:  (-0.8308284156981052, -0.47867020922256326)

E4E4 vs E3E4
Confidence Interval for u1-u2:  (-0.5973006956749015, -0.22627834049508685)

E4E4 vs E2E3
Confidence Interval for u1-u2:  (-0.9213810946183797, -0.4999877664056537)

E3E3 vs E3E4
Confidence Interval for u1-u2:  (0.12741877891129982, 0.35850080983938015)

E3E3 vs E2E3
Confidence Interval for u1-u2:  (-0.20961130338707254, 0.09774106728370754)

E3E4 vs E2E3
Confidence Interval for u1-u2:  (-0.46333321098684943, -0.13445661386719554)


In [104]:
data['APOE'] = data['APOE'].replace({0:'E4E4', 1:'E3E3', 2:'E3E4', 3:'E2E3', 4:'E2E4', 5:'E2E2'})
data['Diagnosis'] = data['Diagnosis'].replace({0:'AD', 1:'CTL', 2:'MCI'})  

# Group the data by 'APOE' and 'Diagnosis' and count the occurrences
grouped_counts = data[['APOE','Diagnosis','Sex']].groupby(['APOE', 'Diagnosis']).count()

# Rename the 'Sex' column to 'Count' as it now represents the count of occurrences
grouped_counts.rename(columns={'Sex': 'Count'}, inplace=True)

# Calculate the sum of each APOE group
total_counts_per_apoe = grouped_counts.groupby('APOE').transform('sum')

# Calculate the percentage for each APOE-Diagnosis combination
grouped_counts['Percentage'] = (grouped_counts['Count'] / total_counts_per_apoe['Count']) * 100

print(grouped_counts)

                Count  Percentage
APOE Diagnosis                   
E2E2 MCI            2  100.000000
E2E3 AD            12   13.636364
     CTL           49   55.681818
     MCI           27   30.681818
E2E4 AD             3   18.750000
     CTL            7   43.750000
     MCI            6   37.500000
E3E3 AD           128   26.175869
     CTL          177   36.196319
     MCI          184   37.627812
E3E4 AD           140   42.813456
     CTL           89   27.217125
     MCI           98   29.969419
E4E4 AD            61   70.114943
     CTL           12   13.793103
     MCI           14   16.091954


Based on the confidence intervals, the differences between certain APOE genotypes suggest the following for Alzheimer's disease (AD) diagnosis:

- E4E4 vs E3E3: The confidence interval does not contain zero, indicating a statistically significant difference between these two genotypes for AD diagnosis. The E4E4 genotype appears to be associated with a higher risk of AD compared to E3E3.
- E4E4 vs E3E4: The confidence interval does not contain zero, suggesting a significant difference between E4E4 and E3E4 genotypes for AD diagnosis. The E4E4 genotype likely has a higher risk of AD than E3E4.
- E4E4 vs E2E3: The confidence interval does not contain zero, implying a significant difference between E4E4 and E2E3 genotypes. The E4E4 genotype is likely associated with a higher risk of AD compared to E2E3.
- E3E3 vs E3E4: The confidence interval does not contain zero, indicating a significant difference between these genotypes for AD diagnosis. The E3E4 genotype may have a higher risk of AD than E3E3.
- E3E3 vs E2E3 and E3E4 vs E2E3: The confidence intervals contain zero, suggesting no statistically significant difference between these genotype pairs for AD diagnosis.


In summary, the presence of the E4 allele, particularly the E4E4 genotype, appears to be associated with a higher risk of Alzheimer's disease compared to other APOE genotypes like E3E3, E3E4, and E2E3.

#### 3. MMSE and AD

In [105]:
data['MMSE_Cat'] = pd.cut(data['MMSE'], bins=[6,19,24.5,30], labels=['low','moderate','high'])

In [106]:
q1 = np.quantile(data['MMSE'], 0)
q2 = np.quantile(data['MMSE'], 0.25)
q3 = np.quantile(data['MMSE'], 0.5)
q4 = np.quantile(data['MMSE'], 0.75)
q5 = np.quantile(data['MMSE'], 1)
print(q1,q2,q3,q4,q5)

6.0 24.0 27.0 29.0 30.0


In [107]:
data['Diagnosis'] = data['Diagnosis'].replace({'AD':0, 'CTL':1, 'MCI':2}) 

In [108]:
data['MMSE_Cat'] = pd.cut(data['MMSE'], bins=[q1-1,q2,q3,q4,q5], labels=['very low','low','moderate','high'])
very_low = data.query('MMSE_Cat == "very low"')
low = data.query('MMSE_Cat == "low"')
moderate = data.query('MMSE_Cat == "moderate"')
high = data.query('MMSE_Cat == "high"')

print("\nVery Low vs Low")
cm = sms.CompareMeans(sms.DescrStatsW(very_low['Diagnosis']), sms.DescrStatsW(low['Diagnosis']))
print("Confidence Interval for u1-u2: ", str(cm.tconfint_diff(usevar='unequal')))

print("\nVery Low vs Moderate")
cm = sms.CompareMeans(sms.DescrStatsW(very_low['Diagnosis']), sms.DescrStatsW(moderate['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nVery Low vs High")
cm = sms.CompareMeans(sms.DescrStatsW(very_low['Diagnosis']), sms.DescrStatsW(high['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nLow vs Moderate")
cm = sms.CompareMeans(sms.DescrStatsW(low['Diagnosis']), sms.DescrStatsW(moderate['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nLow vs High")
cm = sms.CompareMeans(sms.DescrStatsW(low['Diagnosis']), sms.DescrStatsW(high['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))

print("\nModerate vs High")
cm = sms.CompareMeans(sms.DescrStatsW(moderate['Diagnosis']), sms.DescrStatsW(high['Diagnosis']))
print("Confidence Interval for u1-u2: ", cm.tconfint_diff(usevar='unequal'))


Very Low vs Low
Confidence Interval for u1-u2:  (-1.1238439096220614, -0.8651655900144636)

Very Low vs Moderate
Confidence Interval for u1-u2:  (-1.198711274941612, -0.9915617626010502)

Very Low vs High
Confidence Interval for u1-u2:  (-0.9101504593009034, -0.7155530418179178)

Low vs Moderate
Confidence Interval for u1-u2:  (-0.22278888176788636, 0.021525343861748966)

Low vs High
Confidence Interval for u1-u2:  (0.06476560337954113, 0.2985403951381625)

Moderate vs High
Confidence Interval for u1-u2:  (0.19479985816128098, 0.36976967826256)


Based on the confidence intervals comparing different MMSE score bands, this image suggests the following about the relationship between MMSE scores and Alzheimer's disease (AD) diagnosis:

- Very Low vs Low: The confidence interval contains zero, indicating there may not be a statistically significant difference in AD diagnosis between the "Very Low" and "Low" MMSE score bands.
- Very Low vs Moderate: The confidence interval does not contain zero, suggesting a statistically significant difference between the "Very Low" and "Moderate" MMSE bands for AD diagnosis. The "Very Low" band is likely associated with a higher risk of AD compared to the "Moderate" band.
- Very Low vs High: The confidence interval does not contain zero, implying a significant difference between the "Very Low" and "High" MMSE bands in terms of AD diagnosis. The "Very Low" band is probably associated with a higher risk of AD compared to the "High" band.
- Low vs Moderate: The confidence interval contains zero, indicating no statistically significant difference in AD diagnosis between the "Low" and "Moderate" MMSE bands.
- Low vs High: The confidence interval does not contain zero, suggesting a significant difference between the "Low" and "High" MMSE bands for AD diagnosis. The "Low" band is likely associated with a higher risk of AD compared to the "High" band.
- Moderate vs High: The confidence interval does not contain zero, implying a significant difference between the "Moderate" and "High" MMSE bands in terms of AD diagnosis. The "Moderate" band is probably associated with a higher risk of AD compared to the "High" band.


In summary, lower MMSE scores, particularly in the "Very Low" and "Low" bands, appear to be associated with a higher risk of Alzheimer's disease compared to higher MMSE scores in the "Moderate" and "High" bands.

### Hypothesis Testing

In [109]:
data = pd.read_csv('../data/MRI_data_2.csv')
data

,Visit,Site,Sex,Age,APOE,MMSE,lh_bankssts_thickness,rh_bankssts_thickness,lh_caudalanteriorcingulate_thickness,rh_caudalanteriorcingulate_thickness,...,rhCortexVol,lhCerebralWhiteMatterVol,SubCortGrayVol,MaskVol,BrainSegVol-to-eTIV,MaskVol-to-eTIV,lhSurfaceHoles,rhSurfaceHoles,Subject_ID,Diagnosis
0,0,0,0,86.0,0,20.0,2.224,2.249,2.883,2.172,...,171640.3232,184191.9575,40324.0,1361906.0,0.664587,0.977432,24.0,13.0,KPOADC001_1,0
1,1,0,0,86.0,0,16.0,2.140,2.092,2.991,2.038,...,162216.0014,192981.9801,38797.0,1319352.0,0.659338,0.941138,23.0,17.0,KPOADC001_2,0
2,2,0,0,87.0,0,21.0,2.248,2.343,3.088,2.157,...,168541.5319,189605.4005,39722.0,1342921.0,0.660968,0.959201,30.0,24.0,KPOADC001_5,0
3,0,0,0,68.0,0,26.0,1.971,2.018,2.394,2.336,...,182858.7837,199432.8506,45318.0,1515148.0,0.652202,0.976865,22.0,21.0,KPOADC002_1,0
4,1,0,0,68.0,0,25.0,2.066,2.112,2.353,2.285,...,182255.2786,195086.1362,44336.0,1508917.0,0.643059,0.968317,20.0,9.0,KPOADC002_2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,0,5,0,72.0,1,29.0,2.077,2.212,3.181,2.595,...,158168.3125,195236.8035,43989.0,1291996.0,0.694822,1.009026,20.0,14.0,TLSMCI607_1,2
1005,3,5,0,72.0,1,29.0,2.128,2.249,2.709,2.623,...,160279.0441,193091.3118,43329.0,1254851.0,0.697605,0.990604,17.0,16.0,TLSMCI607_1.5,2
1006,0,5,1,73.0,1,28.0,2.146,2.179,2.776,2.353,...,175083.0216,201773.2805,40044.0,1410540.0,0.684031,1.032691,35.0,29.0,TLSMCI608_1,2
1007,3,5,1,73.0,1,28.0,2.068,2.183,2.793,2.405,...,177003.4186,204008.1402,40428.0,1428442.0,0.685443,1.044901,39.0,34.0,TLSMCI608_1.5,2


In [110]:
import pandas as pd
from scipy import stats

data_no_key = data.drop(columns=['Subject_ID'])
ad = data_no_key.query('Diagnosis == 0')
ctl = data_no_key.query('Diagnosis == 1')
# Perform an independent t-test 
t_stat, p_val = stats.ttest_ind(ad, ctl, axis=0)

significant_features = []
# Iterate over the T-statistics and P-values to print out results for each comparison
for i, (t, p) in enumerate(zip(t_stat, p_val)):
    # print(f"Comparison {i}: T-statistic = {t}, P-value = {p}")
    if p < 0.05:
        print(f"  -> Significant difference detected at comparison {i}")
        print(f"Feature {data_no_key.columns[i]} is significantly different between AD and CTL groups")
        significant_features.append(data_no_key.columns[i])

  -> Significant difference detected at comparison 0
Feature Visit is significantly different between AD and CTL groups
  -> Significant difference detected at comparison 1
Feature Site is significantly different between AD and CTL groups
  -> Significant difference detected at comparison 2
Feature Sex is significantly different between AD and CTL groups
  -> Significant difference detected at comparison 3
Feature Age is significantly different between AD and CTL groups
  -> Significant difference detected at comparison 4
Feature APOE is significantly different between AD and CTL groups
  -> Significant difference detected at comparison 5
Feature MMSE is significantly different between AD and CTL groups
  -> Significant difference detected at comparison 6
Feature lh_bankssts_thickness is significantly different between AD and CTL groups
  -> Significant difference detected at comparison 7
Feature rh_bankssts_thickness is significantly different between AD and CTL groups
  -> Significan

/home/fathinah.izzati/miniconda3/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:563: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, axis=axis, **kwds)


In [111]:
print(f"Significant features: {len(significant_features)}")

Significant features: 116
